# Imports

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

---

# Carga de datos

In [6]:
targets_compentencia = pd.read_pickle("../../../data/tp2/ref_hashes_target.pkl")
features_v1 = pd.read_pickle("../features/auctions_train_ventana_1.pkl")
features_v2 = pd.read_pickle("../features/auctions_train_ventana_2.pkl")
features_v3 = pd.read_pickle("../features/auctions_train_ventana_3.pkl")
features_v4 = pd.read_pickle("../features/auctions_train_ventana_4.pkl")
features_v5 = pd.read_pickle("../features/auctions_train_ventana_5.pkl")

In [11]:
ventanas = [features_v1, features_v2, features_v3, features_v4, features_v5]

---

# Miro coincidencias para las ventanas

In [26]:
i = 1
for ventana in ventanas:
    coincidencias = pd.Series(targets_compentencia.index.isin(ventana.index)).value_counts()
    porcentaje = round(100*coincidencias[True] / (coincidencias[True] + coincidencias[False]), 2)
    print("El {}% de los dispositivos de la competencia están en la ventana {}".format(porcentaje, i))
    i += 1

El 68.0% de los dispositivos de la competencia están en la ventana 1
El 69.63% de los dispositivos de la competencia están en la ventana 2
El 73.22% de los dispositivos de la competencia están en la ventana 3
El 75.15% de los dispositivos de la competencia están en la ventana 4
El 76.81% de los dispositivos de la competencia están en la ventana 5


---

# Para cada dispositivo, veo en qué ventanas aparece

In [29]:
apariciones = targets_compentencia.copy()

In [30]:
for ventana_nro in range(1,6):
    apariciones["en_ventana_{}".format(ventana_nro)] = targets_compentencia.index.isin(ventanas[ventana_nro - 1].index)

In [35]:
apariciones.head()

,en_ventana_1,en_ventana_2,en_ventana_3,en_ventana_4,en_ventana_5
ref_hash,,,,,
1000169251625791246,True,True,True,True,True
1000395625957344683,True,True,True,True,True
1003027494996471685,True,True,True,True,True
1006670001679961544,False,False,False,False,True
1007573308966476713,True,True,True,True,True
